## Agents and Tools

In [1]:
# TODO: Complex QA
# question = "Who is the current leader of Japan? What is the largest prime number that is smaller than their age?"
# question = "Find the current population of Japan and then multiply it by 2"

In [2]:
# !pip install langchain langchain-community langchain-openai python-dotenv google-search-results

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
serpapi_api_key = os.environ["SERPAPI_API_KEY"]

MODEL = "gpt-4o-mini"

In [4]:
import os
from typing import Dict, Any, List

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI

# from langchain_community.tools.serpapi import SerpAPIWrapper
# from langchain.tools import CalculatorTool

### Define tool 'Calculator'

In [5]:
import math
import numexpr

def calculator(expression: str) -> str:
    """Calculate expression using Python's numexpr library.

    Expression should be a single line mathematical expression
    that solves the problem.

    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

### Define tool 'Web Search'

In [6]:
from langchain_community.utilities import SerpAPIWrapper

def web_search(query: str) -> str:
    """Perform a web search using SerpAPI and return search results."""
    search = SerpAPIWrapper()
    return search.run(query)

### Create tools and Initialize agent

In [7]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import tool
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
import requests
from sympy import prevprime

# Create tools list
tools = [
    Tool(name="Web Search", func=web_search, description="Searches the web for information."),
    Tool(name="Calculator", func=calculator, description="Evaluates mathematical expressions."),
]

# Initialize the agent
# - Zero-shot means the agent functions on the current action only - it has no memory.
#   It uses the ReAct framework to decide which tool to use, based solely on the tool's description.
llm = ChatOpenAI(model=MODEL, temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

C:\Users\Pavel\AppData\Local\Temp\ipykernel_27372\1935553255.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


### QA without tool

In [8]:
question = "Explain to me the word 'easy-peasy' briefly?"
response = agent.invoke(question)
print(response)
print(response['output'])



> Entering new AgentExecutor chain...
The term "easy-peasy" is an informal expression used to describe something that is very easy or simple to do. It often conveys a sense of lightheartedness or playfulness. To provide a more detailed explanation, I might need to look up its origins or usage examples. 

Action: Web Search  
Action Input: "easy-peasy meaning and origin"  
Observation: Easy peasy lemon squeezy is an elaboration on easy-peasy, likewise meaning “extremely easy or simple.” One of the earliest documented instances of easy-peasy appears in the 1940 American film The Long Voyage Home, used to advise a character to handle a suspicious box with care.
I have gathered information about the term "easy-peasy," including its meaning and some historical context. 

Final Answer: The term "easy-peasy" is an informal expression meaning something that is very easy or simple to do. It is often used in a lighthearted or playful manner. The phrase "easy peasy lemon squeezy" is a variation

### QA wit tool 'Calculator'

In [9]:
question = "37593 times 67"
# question = "37593**(1/5)" for "37593^(1/5)"
# question = "What is 256 multiplied by 13, then divided by 4?"
response = agent.invoke(question)
print(response)
print(response['output'])



> Entering new AgentExecutor chain...
I need to calculate the product of 37593 and 67. 
Action: Calculator
Action Input: 37593 * 67
Observation: 2518731
I now know the final answer.  
Final Answer: 2518731

> Finished chain.
{'input': '37593 times 67', 'output': '2518731'}
2518731


### QA wit tool 'Web Search'

In [10]:
question = "Who is the current leader of Japan?"
# question = "What is the latest breakthrough in AI?"
response = agent.invoke(question)
print(response)
print(response['output'])



> Entering new AgentExecutor chain...
I need to find out who the current leader of Japan is, which is likely the Prime Minister. I'll perform a web search to get the most up-to-date information.  
Action: Web Search  
Action Input: "current Prime Minister of Japan 2023"  
Observation: ['Fumio Kishida type: Member of the House of Representatives of Japan.', 'Fumio Kishida kgmid: /m/03c7pxm.', 'Fumio Kishida born: 1957 (age 67 years), Shibuya, Tokyo, Japan.', 'Fumio Kishida office: Member of the House of Representatives of Japan since 1996.', 'Fumio Kishida party: Liberal Democratic Party.', 'Fumio Kishida children: Shotaro Kishida.', 'Fumio Kishida education: School of Law, Waseda University (1978–1982).', 'Fumio Kishida nationality: Japanese.', 'The current prime minister is Shigeru Ishiba, who succeeded Fumio Kishida on 1 October 2024, following the 2024 Liberal Democratic Party presidential election.', 'Fumio Kishida is a Japanese politician who served as Prime Minister of Japan an

### Complex QA

In [11]:
question = "Who is the current leader of Japan? How old is?"
# question = "Who is the current leader of Japan? What is the largest prime number that is smaller than their age?"
# question = "Find the current population of Japan and then multiply it by 2"
response = agent.invoke(question)
print(response)
print(response['output'])



> Entering new AgentExecutor chain...
I need to find out who the current leader of Japan is and their age. I'll start by searching for the current Prime Minister of Japan. 

Action: Web Search  
Action Input: "current Prime Minister of Japan 2023"  
Observation: ['Fumio Kishida type: Member of the House of Representatives of Japan.', 'Fumio Kishida kgmid: /m/03c7pxm.', 'Fumio Kishida born: 1957 (age 67 years), Shibuya, Tokyo, Japan.', 'Fumio Kishida office: Member of the House of Representatives of Japan since 1996.', 'Fumio Kishida party: Liberal Democratic Party.', 'Fumio Kishida children: Shotaro Kishida.', 'Fumio Kishida education: School of Law, Waseda University (1978–1982).', 'Fumio Kishida nationality: Japanese.', 'The current prime minister is Shigeru Ishiba, who succeeded Fumio Kishida on 1 October 2024, following the 2024 Liberal Democratic Party presidential election.', 'Fumio Kishida is a Japanese politician who served as Prime Minister of Japan and President of the Libe

## Links

### ReAct (Create agent that uses ReAct prompting)

In [12]:
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html

### How to migrate from legacy LangChain agents to LangGraph

In [13]:
# https://python.langchain.com/docs/how_to/migrate_agent/